In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')

torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


device: cuda


100%|██████████| 9912422/9912422 [00:00<00:00, 263816992.15it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 31252758.98it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 151664285.01it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 6258386.59it/s]


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
class Net_0(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)               
        self.conv5 = nn.Conv2d(256, 512, 3)           #input - 7, Output 5, RF
        self.conv6 = nn.Conv2d(512, 1024, 3)          #input - 5, Output 3, RF
        self.conv7 = nn.Conv2d(1024, 10, 3)           #input - 3, Output 1, RF

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_0

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.2577, Accuracy: 8935/10000 (89%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pas

<ipython-input-4-b72feb762c6a>:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Fix Relu before the last layer

class Net_1(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)               
        self.conv5 = nn.Conv2d(256, 512, 3)           #input - 7, Output 5, RF
        self.conv6 = nn.Conv2d(512, 1024, 3)          #input - 5, Output 3, RF
        self.conv7 = nn.Conv2d(1024, 10, 3)           #input - 3, Output 1, RF

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_1

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.0281, Accuracy: 9916/10000 (99%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pas

<ipython-input-5-97bb41efa4e8>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Replace FCs by 1x1x256x10 + GAP

class Net_2(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_2

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.0533, Accuracy: 9837/10000 (98%)
# 20: Test set: Average loss: 0.0283, Accuracy: 9900/10000 (99%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7             [-1, 10, 7, 7]           2,570
         AvgPool2d-8             [-1, 10, 1, 1]               0
Total params: 390,410
Trainable params: 390,410
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.34
Params size (MB): 1.49
Estimated Total Size (MB): 

<ipython-input-6-004b74ddca23>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Remove conv4. max 64 ch

class Net_3(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #input - 28, Output 28, RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)   #input - 14, Output 14, RF
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 10, 1)              #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv3(x)))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_3

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

# 10: Test set: Average loss: 0.1315, Accuracy: 9608/10000 (96%)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4           [-1, 64, 14, 14]          36,928
         MaxPool2d-5             [-1, 64, 7, 7]               0
            Conv2d-6             [-1, 10, 7, 7]             650
         AvgPool2d-7             [-1, 10, 1, 1]               0
Total params: 56,394
Trainable params: 56,394
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.79
Params size (MB): 0.22
Estimated Total Size (MB): 1.01
-------------------------------------------------------------

<ipython-input-7-1c5ab9dea573>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Add BN

class Net_4(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)     #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)   #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(64)
        # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 10, 1)              #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.bn3(self.conv3(x))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_4

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#epoch: 10
#loss=0.03565816953778267 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.78it/s]
#Test set: Average loss: 0.1311, Accuracy: 9607/10000 (96%)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
       BatchNorm2d-4           [-1, 64, 28, 28]             128
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6           [-1, 64, 14, 14]          36,928
       BatchNorm2d-7           [-1, 64, 14, 14]             128
         MaxPool2d-8             [-1, 64, 7, 7]               0
            Conv2d-9             [-1, 10, 7, 7]             650
        AvgPool2d-10             [-1, 10, 1, 1]               0
Total params: 56,714
Trainable params: 56,714
Non-trainable params: 0
-------------------------------------------------

<ipython-input-8-c3895eceecbc>:25: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Add BN to Net_2

class Net_5(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)                    

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x)))))))
        x = self.pool2(F.relu(self.bn4(self.conv4(F.relu(self.bn3(self.conv3(x)))))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_5

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

#epoch: 10
#loss=0.019351879134774208 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.98it/s]
#Test set: Average loss: 0.0452, Accuracy: 9850/10000 (98%)

#epoch: 19
#loss=0.0030171473044902086 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.02it/s]
#Test set: Average loss: 0.0286, Accuracy: 9914/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
            Conv2d-3           [-1, 64, 28, 28]          18,496
       BatchNorm2d-4           [-1, 64, 28, 28]             128
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
       BatchNorm2d-7          [-1, 128, 14, 14]             256
            Conv2d-8          [-1, 256, 14, 14]         295,168
       BatchNorm2d-9          [-1, 256, 14, 14]             512
        MaxPool2d-10            [-1, 256, 7, 7]               0
           Conv2d-11             [-1, 10, 7, 7]           2,570
        AvgPool2d-12             [-1, 10, 1, 1]        

<ipython-input-9-e21bbe09ed1e>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Add dropout

class Net_6(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.05)
        self.dropout2 = nn.Dropout(0.05)
        self.dropout3 = nn.Dropout(0.05)
        self.dropout4 = nn.Dropout(0.05)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_6

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#loss=0.016568930819630623 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.46it/s]
#Test set: Average loss: 0.0456, Accuracy: 9866/10000 (99%)

#epoch: 20
#loss=0.008861390873789787 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.52it/s]
#Test set: Average loss: 0.0351, Accuracy: 9891/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

<ipython-input-10-c24d52b133e8>:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Larger dropout

class Net_7(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        return F.log_softmax(x)

Net = Net_7

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


#epoch: 10
#loss=0.007140759844332933 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.26it/s]
#Test set: Average loss: 0.0569, Accuracy: 9839/10000 (98%)

#epoch: 20
#loss=0.024968238547444344 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.36it/s]
#Test set: Average loss: 0.0469, Accuracy: 9878/10000 (99%)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

<ipython-input-11-c908b5d6768b>:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
# Use cross_entropy instead of log_softmax

class Net_8(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        # return F.log_softmax(x)
        return x

Net = Net_8

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


from tqdm import tqdm
def train_ce(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test_ce(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
  
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(f'epoch: {epoch}')
    train_ce(model, device, train_loader, optimizer, epoch)
    test_ce(model, device, test_loader)

#stopped early
# 59% accuracy at epoch 8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<ipython-input-12-f93bebd886db>:46: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]             512
          Dropout-13          [-1, 256, 14, 14]               0
        MaxPool2d-14      

loss=1.9399456977844238 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.91it/s]



Test set: Average loss: 2.0766, Accuracy: 4179/10000 (42%)

epoch: 2


...


Test set: Average loss: 1.8707, Accuracy: 5911/10000 (59%)

epoch: 11


loss=1.8039277791976929 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.88it/s]



Test set: Average loss: 1.8696, Accuracy: 5922/10000 (59%)

epoch: 12


loss=1.838909387588501 batch_id=216:  46%|████▋     | 217/469 [00:10<00:12, 20.79it/s]


KeyboardInterrupt: ignored

In [ ]:
# Use cross_entropy instead of log_softmax, use Adam

class Net_9(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)   #input - 28, Output 28, RF
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.bn2 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)               
        
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) #input - 14, Output 14, RF
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(256, 10, 1)            #input - 7, Output 7, RF
        self.gap = nn.AvgPool2d(7)          

        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.dropout3 = nn.Dropout(0.1)
        self.dropout4 = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.dropout2(x)

        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = self.dropout3(x)
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.dropout4(x)

        x = self.pool2(x)
        
        x = F.relu(self.conv5(x))
        x = self.gap(x)

        x = x.view(-1, 10)
        # return F.log_softmax(x)
        return x

Net = Net_9

%pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'device: {device}')
model = Net().to(device)
summary(model, input_size=(1, 28, 28))


from tqdm import tqdm
def train_ce(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test_ce(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
  
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 21):
    print(f'epoch: {epoch}')
    train_ce(model, device, train_loader, optimizer, epoch)
    test_ce(model, device, test_loader)

#stopped early
# 59% accuracy at epoch 8



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
device: cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
       BatchNorm2d-2           [-1, 32, 28, 28]              64
           Dropout-3           [-1, 32, 28, 28]               0
            Conv2d-4           [-1, 64, 28, 28]          18,496
       BatchNorm2d-5           [-1, 64, 28, 28]             128
           Dropout-6           [-1, 64, 28, 28]               0
         MaxPool2d-7           [-1, 64, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]          73,856
       BatchNorm2d-9          [-1, 128, 14, 14]             256
          Dropout-10          [-1, 128, 14, 14]               0
           Conv2d-11          [-1, 256, 14, 14]         295,168
      BatchNorm2d-12          [-1, 256, 14, 14]        

loss=1.429561734199524 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.57it/s]



Test set: Average loss: 1.4627, Accuracy: 3963/10000 (40%)

epoch: 2


loss=1.3300385475158691 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.26it/s]



Test set: Average loss: 1.4318, Accuracy: 3973/10000 (40%)

epoch: 3


...


Test set: Average loss: 1.4140, Accuracy: 3993/10000 (40%)

epoch: 10


loss=1.5272210836410522 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.86it/s]



Test set: Average loss: 1.4089, Accuracy: 3989/10000 (40%)

epoch: 11


loss=1.5360075235366821 batch_id=135:  29%|██▉       | 136/469 [00:06<00:15, 21.55it/s]


KeyboardInterrupt: ignored

In [ ]:


optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print(f'epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch: 1


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-14-f3131be26054>:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.089035265147686 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.74it/s]



Test set: Average loss: 0.2352, Accuracy: 9413/10000 (94%)

epoch: 2


loss=0.06830409914255142 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.36it/s]



Test set: Average loss: 0.1522, Accuracy: 9597/10000 (96%)

epoch: 3


...


Test set: Average loss: 0.0490, Accuracy: 9853/10000 (99%)

epoch: 19


loss=0.008810898289084435 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.86it/s]



Test set: Average loss: 0.0353, Accuracy: 9905/10000 (99%)

epoch: 20


loss=0.024968238547444344 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.36it/s]



Test set: Average loss: 0.0469, Accuracy: 9878/10000 (99%)

